# CRISM to CTX reprojection

In [ ]:
import rasterio
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
from rasterio import shutil as rio_shutil
import os
from utils import copy_crs_from_ctx

def reproject(ctx_path, crism_path, output_dir):
    with rasterio.open(crism_path, driver='ENVI') as crism_src, rasterio.open(ctx_path, driver='ISIS3') as ctx_src:

        # Get CRS, bounds, and resolution from the CTX cube
        ctx_crs = ctx_src.crs
        ctx_bounds = ctx_src.bounds
        ctx_transform = ctx_src.transform
        ctx_width, ctx_height = ctx_src.width, ctx_src.height

        # Set up WarpedVRT parameters to match the CTX image
        vrt_options = {
            'crs': ctx_crs,                          # Use CRS of CTX
            'transform': ctx_transform,              # Use transformation matrix of CTX
            'width': ctx_width,                      # Match dimensions of CTX
            'height': ctx_height,
            'resampling': Resampling.cubic       # Choose resampling method
        }

        # Create a virtual warped dataset to match the CTX cube
        with WarpedVRT(crism_src, **vrt_options) as vrt:
            data = vrt.read()

            for _, window in vrt.block_windows():
                data = vrt.read(window=window)
            # Now, 'vrt' has the CRISM data in the same projection and resolution as the CTX image
            # You can save the warped dataset if needed
            output_path = os.path.join(output_dir, modify_filename(os.path.split(crism_path)[-1]))
            rio_shutil.copy(vrt, output_path, driver='ENVI', strict=True)

    copy_crs_from_ctx(output_path, ctx_path)


def modify_filename(file_name):
    base_name = os.path.splitext(file_name)[0]
    return f"{base_name}.rp.img"
    

In [ ]:
crism_path = ''
ctx_path = ''
output_dir = ''

reproject(ctx_path, crism_path, output_dir)

### Rename files to upper case

In [ ]:
from utils import files_in_dir_to_upper_case

crism_path = ""

files_in_dir_to_upper_case(crism_path)

### Batch CRS reprojection

In [ ]:
import os
from utils import holden_crater_pairs

evc_folder = ""
ctx_folder = ""
output_dir = ""

os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(evc_folder):
    try:
        if file.endswith(".img"):
            
            evc_base_name = os.path.splitext(file)[0]

            ctx_base_name = holden_crater_pairs[evc_base_name]

            evc_path = os.path.join(evc_folder, file)
            ctx_path = os.path.join(ctx_folder, f"{ctx_base_name}.cub")

            reproject(ctx_path, evc_path, output_dir)

            print(f"Finished: {file}")

    except Exception as e:
        print(f"Error processing {file}: {e}")

Updated coordinate system in: G:/thesis_data_arch/raw_data/holden/crism/evc/reprojected_1\FRT0000474A_07_IFEVCJ_MTR3.rp.hdr
Finished: FRT0000474A_07_IFEVCJ_MTR3.img
Updated coordinate system in: G:/thesis_data_arch/raw_data/holden/crism/evc/reprojected_1\FRT00004F2F_07_IFEVCJ_MTR3.rp.hdr
Finished: FRT00004F2F_07_IFEVCJ_MTR3.img
Updated coordinate system in: G:/thesis_data_arch/raw_data/holden/crism/evc/reprojected_1\FRT00006246_07_IFEVCJ_MTR3.rp.hdr
Finished: FRT00006246_07_IFEVCJ_MTR3.img
Updated coordinate system in: G:/thesis_data_arch/raw_data/holden/crism/evc/reprojected_1\FRT00009172_07_IFEVCJ_MTR3.rp.hdr
Finished: FRT00009172_07_IFEVCJ_MTR3.img
Updated coordinate system in: G:/thesis_data_arch/raw_data/holden/crism/evc/reprojected_1\FRT00009D17_07_IFEVCJ_MTR3.rp.hdr
Finished: FRT00009D17_07_IFEVCJ_MTR3.img
Updated coordinate system in: G:/thesis_data_arch/raw_data/holden/crism/evc/reprojected_1\FRT0000A98D_07_IFEVCJ_MTR3.rp.hdr
Finished: FRT0000A98D_07_IFEVCJ_MTR3.img
Updated co

# CASSIS reprojection

### CaSSIS to CTX reprojection

In [ ]:
import rasterio
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
from rasterio import shutil as rio_shutil
import os

def reproject_cassis(ctx_path, cassis_path, output_dir):
    with rasterio.open(cassis_path, driver='ISIS3') as cassis_src, rasterio.open(ctx_path, driver='ISIS3') as ctx_src:

        # Get CRS, bounds, and resolution from the CTX cube
        ctx_crs = ctx_src.crs
        ctx_bounds = ctx_src.bounds
        ctx_transform = ctx_src.transform
        ctx_width, ctx_height = ctx_src.width, ctx_src.height

        # Set up WarpedVRT parameters to match the CTX image
        vrt_options = {
            'crs': ctx_crs,                          # Use CRS of CTX
            'transform': ctx_transform,              # Use transformation matrix of CTX
            'width': ctx_width,                      # Match dimensions of CTX
            'height': ctx_height,
            'resampling': Resampling.cubic       # Choose resampling method
        }

        # Create a virtual warped dataset to match the CTX cube
        with WarpedVRT(cassis_src, **vrt_options) as vrt:
            data = vrt.read()

            for _, window in vrt.block_windows():
                data = vrt.read(window=window)
            # Now, 'vrt' has the CRISM data in the same projection and resolution as the CTX image
            # You can save the warped dataset if needed
            output_path = os.path.join(output_dir, modify_filename(os.path.split(cassis_path)[-1]))
            rio_shutil.copy(vrt, output_path, driver='ISIS3', strict=True)


def modify_filename(file_name):
    base_name = os.path.splitext(file_name)[0]
    return f"{base_name}.rp.cub"
    

In [ ]:
cassis_path = ''
ctx_path = ''
output_dir = ''

reproject_cassis(ctx_path, cassis_path, output_dir)

## Reproject CRISM to CASSIS

In [10]:
import rasterio
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
from rasterio import shutil as rio_shutil
import os

def reproject_crism_to_cassis(crism_path, cassis_path, output_dir):
    with rasterio.open(cassis_path, driver='ISIS3') as cassis_src, rasterio.open(crism_path, driver='GTiff') as crism_src:

        # Get CRS, bounds, and resolution from the CTX cube
        cassis_crs = cassis_src.crs
        cassis_bounds = cassis_src.bounds
        cassis_transform = cassis_src.transform
        cassis_width, cassis_height = cassis_src.width, cassis_src.height

        # Set up WarpedVRT parameters to match the CTX image
        vrt_options = {
            'crs': cassis_crs,                          # Use CRS of CTX
            'transform': cassis_transform,              # Use transformation matrix of CTX
            'width': cassis_width,                      # Match dimensions of CTX
            'height': cassis_height,
            'resampling': Resampling.cubic       # Choose resampling method
        }

        # Create a virtual warped dataset to match the CTX cube
        with WarpedVRT(crism_src, **vrt_options) as vrt:
            data = vrt.read()

            for _, window in vrt.block_windows():
                data = vrt.read(window=window)
            # Now, 'vrt' has the CRISM data in the same projection and resolution as the CTX image
            # You can save the warped dataset if needed
            output_path = os.path.join(output_dir, modify_filename(os.path.split(crism_path)[-1]))
            rio_shutil.copy(vrt, output_path, driver='GTiff', strict=True)


def modify_filename(file_name):
    base_name = os.path.splitext(file_name)[0]
    return f"{base_name}.tif"
    

### CRISM to CaSSIS batch reprojection

In [ ]:
import os
from utils import jezero_crism_cassis_pairs

evc_folder = ""
cassis_folder = ""
output_dir = ""

os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(evc_folder):
    try:
        if file.endswith(".tif"):
            
            evc_base_name = os.path.splitext(file)[0]

            cassis_base_name = jezero_crism_cassis_pairs[evc_base_name]

            evc_path = os.path.join(evc_folder, file)
            cassis_path = os.path.join(cassis_folder, f"{cassis_base_name}.cubeit.cub")

            reproject_crism_to_cassis(evc_path, cassis_path, output_dir)

            print(f"Finished: {file}")

    except Exception as e:
        print(f"Error processing {file}: {e}")

Finished: FRT00005C5E_07_IFEVCJ_MTR3.tif


### Projection WKT fix

In [4]:
import rasterio

ctx_path = 'G:/thesis_data_arch/raw_data/jezero/ctx/equirectangular/F04_037396_1985_XN_18N282W.cal.er.4.cub'

with rasterio.open(ctx_path, driver='ISIS3') as ctx_src:
    print(ctx_src.crs.to_wkt())

PROJCS["Equirectangular Mars",GEOGCS["GCS_Mars",DATUM["D_Mars",SPHEROID["Mars_localRadius",3394839.813,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",15],PARAMETER["central_meridian",180],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [5]:
import rasterio
import re

def modify_mars_local_radius(ctx_path, new_radius):
    """
    Modifies the Mars_localRadius value in the WKT of a CTX file.
    Args:
        ctx_path (str): Path to the CTX file.
        new_radius (float): New Mars_localRadius value to set.
    """
    with rasterio.open(ctx_path, 'r+', driver='ISIS3') as ctx_src:
        wkt = ctx_src.crs.to_wkt()
        data = ctx_src.read()
        # Replace Mars_localRadius value using regex
        modified_wkt = re.sub(r'Mars_localRadius",([0-9.]+)', f'Mars_localRadius",{new_radius}', wkt)
        # Update the CRS with the modified WKT
        # ctx_src.update_tags(1, coordinate_system_string=f"{{{modified_wkt}}}")
        profile = ctx_src.profile
        profile['crs'] = modified_wkt
        # ctx_src.crs = modified_wkt
    with rasterio.open(ctx_path, 'w', **profile) as dst:
        dst.write(data)
        print("Updated WKT:", modified_wkt)

# Example usage
ctx_path = 'G:/thesis_data_arch/raw_data/jezero/ctx/equirectangular/F04_037396_1985_XN_18N282W.cal.er.4.cub'
# ctx_path = './ctx/F04_037396_1985_XN_18N282W.cal.er.2.cub'
new_radius = 3394839.813  # Example new radius value
modify_mars_local_radius(ctx_path, new_radius)

Updated WKT: PROJCS["Equirectangular Mars",GEOGCS["GCS_Mars",DATUM["D_Mars",SPHEROID["Mars_localRadius",3394839.813,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",15],PARAMETER["central_meridian",180],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
